<a href="https://colab.research.google.com/github/DatacationOrg/chatbot-workshop/blob/main/Datacation_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile requirements.txt
chainlit==2.2.1

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

In [3]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
changed 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [4]:
%%writefile app.py
import chainlit as cl

@cl.on_message
async def on_message(message: cl.Message):
    await cl.Message(content=message.content).send()

Overwriting app.py


In [5]:
import subprocess

chainlit_process = None
if chainlit_process:
    chainlit_process.kill()
tunnel_process = None
if tunnel_process:
    tunnel_process.kill()
output_file = "output.log"
out_log = open(output_file, "wb")
chainlit_process = subprocess.Popen(
    ["chainlit", "run", "app.py", "--host", "0.0.0.0", "--port", "8000"],
    stdout=out_log,
    stderr=subprocess.STDOUT,
)
tunnel_process = subprocess.Popen(
    ["lt", "--port", "8000"], stdout=out_log, stderr=subprocess.STDOUT
)
!sleep 5s # Allow for some time to properly boot

In [10]:
assert chainlit_process.poll() is None
assert tunnel_process.poll() is None
!cat $output_file
!echo "Tunnel Password: $(curl -s https://loca.lt/mytunnelpassword)"

your url is: https://fruity-beans-drum.loca.lt
2025-02-27 13:49:52 - Your app is available at http://0.0.0.0:8000
Tunnel Password: 34.169.173.84
